In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import pickle

In [2]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 49.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-

# Audf

In [3]:
with open('../input/entities/entityAu.pkl', 'rb') as f:
    entitiesA = pickle.load(f)

In [4]:
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

Audf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})



for i in entitiesA:
    
 dc={}
 P=[]
 D=[]
 G=[]
 O=[]
 for entity in i:
    if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        
        
        
 for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
 Per.append(P)
 Date.append(D)
 ORG.append(O)
 GPE.append(G)
            
Audf["Date-ontonotes"]=Date
Audf["PERSON-ontonotes"]=Per
Audf["GPE-ontonotes"]=GPE
Audf["ORG-ontonotes"]=ORG    

In [5]:
Audf

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,NaN,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]"
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864]
2,NaN,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]"
3,NaN,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]"
4,NaN,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]"
...,...,...,...,...,...
2131,NaN,"[Geo . Little, Geo . B. Ayer]","[Raleigh, N.C., Wake County, N.C, United State...",[],"[May 26, July 14]"
2132,NaN,[Birnie],[],[],[July 30th 1867]
2133,NaN,[William Birnie],"[North Carolina, Robeson County, United States]",[],[30 day of July]
2134,NaN,"[A.L . Price, James Fulton, John D Conoley, Al...","[North Carolina, New Hanover County, New Hanov...",[Fulton Price],"[May 29 1865, 15th Sept 1865, 15th September A..."


In [6]:
G=[]
for i in Audf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
Audf["postDate-ontonotes"]=GF

In [7]:
G=[]
for i in Audf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

Audf["postDate-ontonotes"]=G

In [8]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta","Aug.",'Agusut']:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh',"Sep.","Sept."]:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The","Dec"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber",'Dec.']:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY",'Jan.']:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta","Aug.","Agusut"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This ","Sept.","Sep."]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber","Dec."]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY","Jan."]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb","Feb."]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth")|(dates[0]=="The 9th ") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ")|(dates[0]=="Seventeenth") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [9]:
Audf["Dates"]=NG

In [10]:
Audf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates
0,NaN,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865]
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[]
2,NaN,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865]
3,NaN,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[]
4,NaN,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867]


In [11]:
Audf["text"]=range(2136)

In [12]:
len(NG)

2136

In [13]:
PL=[]
for i in NG:
    for j in i:
        PL.append(j)
len(PL)

3587

In [14]:
def uniqueName(iterable):
    seen = set()
    result = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
NGG=[]
for i in NG:
    i=uniqueName(i)
    NGG.append(i)

In [15]:
PL=[]
for i in NGG:
    for j in i:
        PL.append(j)
len(PL)

2887

In [16]:

for i in NGG:
    for j in i:
        if j=="29-May-1865":
            i.remove(j)

In [17]:
for i in range(len(NGG)):
    if len(NGG[i])==2:
        NGG[i].remove(NGG[i][1])
    elif (len(NGG[i])>2):
        v=NGG[i][1]
        del(NGG[i][0:])
        NGG[i]=[v]
        

****

# dflarge

In [18]:
with open('../input/entities/entityL.pkl', 'rb') as f:
    entitiesL= pickle.load(f)

In [19]:
len(entitiesL)

2136

In [20]:
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in entitiesL:
    dc={}
    P=[]
    L=[]
    G=[]
    O=[]
    for entity in i :

                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                       
        
      
    for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
    Per.append(P)
    ORG.append(O)
    LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG   

In [21]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,NaN,"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,NaN,"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,NaN,[God],"[State of, North Carolina, United States]",[Constitution]
4,NaN,"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"


# final df

In [22]:
Audf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
Audf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
Audf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]

In [23]:
Audf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge
0,0,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,1,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[],[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,2,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865],"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,3,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[],[God],"[State of, North Carolina, United States]",[Constitution]
4,4,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867],"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"


In [24]:
import os 
os.getcwd()

'/kaggle/working'

In [25]:
Audf.to_pickle("data.pkl")

In [26]:
with open('../input/audfff/data.pkl', 'rb') as f:
    data= pickle.load(f)

In [27]:
data

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge
0,0,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,1,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[],[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,2,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865],"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,3,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[],[God],"[State of, North Carolina, United States]",[Constitution]
4,4,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867],"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"
...,...,...,...,...,...,...,...,...,...,...
2131,2131,"[Geo . Little, Geo . B. Ayer]","[Raleigh, N.C., Wake County, N.C, United State...",[],"[May 26, July 14]",[],[],"[Geo . Little, Geo, B. Ayer, Hugo Hillebrandt]","[Raleigh, N.C., Wake County, State of N.C, Uni...",[]
2132,2132,[Birnie],[],[],[July 30th 1867],[30-July-1867],[30-July-1867],[],[],[]
2133,2133,[William Birnie],"[North Carolina, Robeson County, United States]",[],[30 day of July],[],[],"[William Birnie, God, Wm Birnie, W.A]","[State of, North Carolina, Robeson County, Uni...",[Constitution]
2134,2134,"[A.L . Price, James Fulton, John D Conoley, Al...","[North Carolina, New Hanover County, New Hanov...",[Fulton Price],"[May 29 1865, 15th Sept 1865, 15th September A...","[29-May-1865, 15-Sept-1865, 15-September-1865,...","[29-May-1865, 15-Septembre-1865, 15-Septembre-...","[Price, James Fulton, Sgd W . N. Peden, Miles ...","[State of North Carolina, New Hanover County, ...","[Fulton Price, Confederate Government, Court o..."


# Separate Oath maker and witness name

In [28]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

In [29]:
newdata["transcription_text"][500]

'[[image - eagle]]\r\nAMNESTY OATH. \r\n\r\nMississippi City Sept 12th 1865.\r\n\r\nI, Redick Cruthirds do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\n\r\nRedick his X mark Cruthirds\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 12 day of Sept\r\n1865, before me,\r\nJ H Mathews Capt\r\n66th US Col Infantry &\r\nProvost Marshal.\r\n\r\nDESCRIPTION:\r\nAge 51 years\r\nHeight 5 feet 8 inches\r\nColor of Eyes Blue\r\nColor of Hair Dark\r\nCharacteristics\r\n\r\nHEADQUARTERS Port Mississippi City Miss\r\nSept 12th 1865\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, w

In [30]:
newdata["transcription_text"][5]

'I Sarah E Dough, of Currituck county, State of North  Carolina do solemnly swear or affirm in presence of Almighty God, that I will hence forth faithfully support, protect and defend the Constitution of the United States and the Union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slave. So help me God\r\n\r\nSarah. E. Dough\r\nsworn and subscribed to this the 1^st day of {Dec.\r\nA.D. 1866, before\r\n\r\nIsaac C. Merkins, J.P.'

In [31]:
newdata["transcription_text"][10]

'[D 93 ENCL]\r\n\r\nI William S Skarren hereby agree to serve in the Bureau of Refugees Freedmen and Abandoned Lands in the Capacity of Clerk, to perform all duties pertaining to that office. And that I will accept in payment for said service $50 fifty dollars per month for each and every month I shall continue to perform the services above stated.\r\nNovember 4th, 1867\r\n\r\nRichard Dillon  seal\r\nCapt V.R.C. Bvt Maj U.S.V. A.S.A.C.\r\n\r\nWilliam E. Skarren  seal\r\n\r\nSigned Sealed & Delivered\r\nin presence of \r\nJohn N. Niles\r\nA.A. Surg USA\r\n     '

In [32]:
newdata["transcription_text"][11]

'[D93 ENCL]\r\n\r\nI William E Skarren hereby agree to serve the Bureau of Refugees Freedmen and Abandoned Lands in the Capacity of Clerk, to perform all duties pertaining to the Office, and that I will accept in payment for said Service $50 fifty dollars  per month, for each and every month I should continue to perform the services above stated.\r\nNovember 4th 1867\r\n\r\nRichard Dillon  seal\r\nCapt. V.R.C. Bvt. Maj U.S.V. A.S.A.C.\r\nWilliam E. Skarren  seal\r\n\r\nSigned Sealed and Delivered\r\nin presence of\r\nJohn N. Niles \r\nA.A. Surg US.A\r\n\r\n'

In [33]:
all_text=newdata["transcription_text"]
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^`{|}~'

In [34]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] 
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:01<00:00, 2130.48it/s]


In [35]:
all_text[0]

'I , I C Meekins , of Currituck County , State of North Carolina , solemnly swear affirm , presence of Almighty God , I henceforth faithfully support , protect defend Constitution of United States Union of States thereunder ; I , like manner , abide faithfully support law proclamation made existing rebellion reference emancipation of slave . So help God . Isaac C. Meekins Sworn subscribed 16 day of Sept A.D. , 1865 , E H Walker , J.P . It hereby certified true copy of original oath taken subscribed I C Meekins E H Walker , J.P. John B Etheridge , J.P .'

# City with regex

In [36]:
City=[]
c=[]
for i in all_text:

    city=re.search(r'[A-Za-z]\w*\s[A-Za-z]\w*.City' ,i)
    
    c.append(city)


for i in c:
    if i !=None:
        City.append(i.group())
City[:10]

['JP Of City',
 'MAYOR Of City',
 'MAYOR Of City',
 'of Albany City',
 'office The City',
 'At Mississippi City',
 'subscribed Mississippi City',
 'At Miss City',
 'Post Miss City',
 'At Miss City']

In [37]:
all_text[1]

'Department of Virginia North Carolina , 1864. taken subscribed OATH PAROLE required General Orders No . 49 , Loyal Citizen of United States , residing State ever rebellion , sympathized seceding States .'

In [38]:
all_text[500]

'AMNESTY OATH . Mississippi City Sept 12th 1865 . I , Redick Cruthirds solemnly swear , presence of Almighty God , I henceforth faithfully support , protect defend Constitution of United States union of States thereunder ; I , like manner , abide faithfully support law proclamation made existing rebellion reference emancipation of slave . SO HELP ME GOD . Redick X mark Cruthirds Sworn subscribed Mississippi City 12 day of Sept 1865 , , J H Mathews Capt 66th US Col Infantry Provost Marshal . DESCRIPTION : Age 51 year Height 5 foot 8 inch Color of Eyes Blue Color of Hair Dark Characteristics HEADQUARTERS Port Mississippi City Miss Sept 12th 1865 I hereby certify , Amnesty Oath prescribed Proclamation of President of United States , duly administered Redick Cruthirds of Harrison County order , 12 day of Sept 1865 . Frank E. Miller Major 66th U.S.C.I . Commanding .'

In [39]:
all_text[500]

'AMNESTY OATH . Mississippi City Sept 12th 1865 . I , Redick Cruthirds solemnly swear , presence of Almighty God , I henceforth faithfully support , protect defend Constitution of United States union of States thereunder ; I , like manner , abide faithfully support law proclamation made existing rebellion reference emancipation of slave . SO HELP ME GOD . Redick X mark Cruthirds Sworn subscribed Mississippi City 12 day of Sept 1865 , , J H Mathews Capt 66th US Col Infantry Provost Marshal . DESCRIPTION : Age 51 year Height 5 foot 8 inch Color of Eyes Blue Color of Hair Dark Characteristics HEADQUARTERS Port Mississippi City Miss Sept 12th 1865 I hereby certify , Amnesty Oath prescribed Proclamation of President of United States , duly administered Redick Cruthirds of Harrison County order , 12 day of Sept 1865 . Frank E. Miller Major 66th U.S.C.I . Commanding .'

In [40]:
all_text[5]

'I Sarah E Dough , of Currituck county , State of North Carolina solemnly swear affirm presence of Almighty God , I hence forth faithfully support , protect defend Constitution of United States Union of States thereunder ; I , like manner , abide faithfully support law proclamation made existing rebellion reference emancipation of slave . So help God Sarah . E. Dough sworn subscribed 1st day of Dec. A.D. 1866 , Isaac C. Merkins , J.P .'

In [41]:
all_text[1000]

'I John Bowers solemnly swear I carefully read Amnesty Proclamation issued ANDREW JOHNSON , President of United States of America , May 29th , 1865 , I excepted benefit of Proclamation one of fourteen exception therein made , except John E. Bowers Sworn subscribed Charleston SC 31st day of August 1865 . Levi Stuber Major 47th Pa. Vols . Pro . Marshal . No . 923 United States of America . I , John Bowers of County City of Charleston State of South Ca solemnly swear presence of Almighty God , I henceforth faithfully support defend Constitution of United States , Union of States thereunder , I , like manner , abide faithfully support law proclamation made existing rebellion reference Emancipation of slaves–SO HELP ME GOD . John E. Bowers Sworn subscribed Charleston SC 31st day of August 1865 . Levi Stuber Major 47th Pa. Vols . Pro . Marshal . The named Dark complexion Dark hair , Grey eye ; 5 foot 8 inch high 54 year ; profession Auctioneer Copy Levi Stuber Major 47th Pa. Vols . Pro Marsh

In [42]:
all_text[1900]

'UNITED STATES OF AMERICA . TEST OATH . I , W.C. Arthur solemnly swear I never voluntarily borne arm United States since I citizen thereof ; I voluntarily given aid , countenance , counsel , encouragement person engaged armed hostility thereto ; I neither sought , accepted , attempted exercise function of office whatever , authority , pretended authority , hostility United States ; I yielded voluntary support pretended government , authority , power , constitution within United States , hostile inimical thereto . And I swear , affirm , , best of knowledge ability , I support defend Constitution of United States enemy , foreign domestic ; I bear true faith allegiance ; I take obligation freely , without mental reservation purpose of evasion ; I well faithfully discharge duty of office I enter : So help God . W.C. Arthur SWORN TO AND SUBSCRIBED , Montgomery , Alabama , 1st day of August 1867 S C Greene Captain 24th Infty Bvt Maj USA . A.A.A.G .'

In [43]:
all_text[248]

'No . Office Provost Marshal , Parish of Orleans , DEPARTMENT OF THE GULF . New Orleans , 1865 . I late C. S. solemnly swear presence of Almighty God , I henceforth faithfully support , protect defend Constitution of United States Union of States thereunder , I , like manner , abide faithfully support Acts of Congress passed existing rebellion reference slave , long far repealed , modified , held void Congress decision of Supreme Court , I , like manner , abide faithfully support Proclamations of President made existing rebellion , reference slave , long far modified declared void decision of Supreme Court . So help God . Sworn Subscribed Before day of 1865 . J P Evans The named complexion , hair , eye , aged year , foot inch high . Doherty Capt . 16th Indiana Vols. , Asst . Provost Marshal .'